In [ ]:
from lxml import html
import requests

stream_id='36'
url = "https://www.cbcsport.az/"
response = requests.get(url)
response.raise_for_status()
page_content = response.content
tree = html.fromstring(page_content)
elements = tree.xpath("(//div[contains(@class, 'swiper-slide')][1]//ul/li)")

programs={}
for li in elements:
    time_element = li.xpath(".//span[contains(@class, 'd-flex align-items-center')]/text()")
    time = time_element[0].strip() if time_element else "No time found"
    content_element = li.xpath(".//p/text()")
    content = content_element[0].strip() if content_element else "No content found"
    # print(f"Time: {time}, Content: {content}")
    programs[time]=content
daily_programs[stream_id]=programs

In [18]:
daily_programs

{'36': {'02:00': 'İtaliya A Seriyası, 14-cü tur: Udineze – Cenoa',
  '05:00': 'İtaliya A Seriyası, 14-cü tur: Fiorentina – İnter',
  '09:30': 'CBC Sport İnfo (canlı)',
  '10:00': 'Ay həkim (premyera)',
  '12:00': 'CBC Sport İnfo (canlı)',
  '12:30': 'Start (canlı)',
  '13:30': 'İtaliya A Seriyası, 14-cü turun oyunlarının icmalı (yeni)',
  '14:30': 'CBC Sport İnfo (canlı)',
  '15:00': 'Futbol, Azərbaycan Kuboku, 1/8 final. Qaradağ Lökbatan – Səbail (canlı)',
  '17:00': 'CBC Sport İnfo (canlı)',
  '18:00': 'Futbol, Azərbaycan Kuboku, 1/8 final. Turan Tovuz – Sabah (canlı)',
  '20:00': 'Misli vaxtı (yeni)',
  '20:15': 'Misli Premyer Liqasının icmalı',
  '21:15': 'İtaliya A Seriyası, 14-cü turun oyunlarının icmalı',
  '23:45': 'Futbol, Azərbaycan Kuboku, 1/8 final. Qaradağ Lökbatan – Səbail'}}

In [20]:
def clean_dict(dict):
    for key, value in dict.items():
        cleaned_value = value.split('.', 1)[0].strip()  
        cleaned_value = ' '.join(cleaned_value.split())  
        dict[key] = cleaned_value
    return dict

def insert_code_to_programs(daily_programs):
    missing_dict = {}
    insert_programs_query = "INSERT INTO programs (name, stream_id) VALUES (%s, %s)"

    for stream in daily_programs:
        try:
            missing = set()
            select_programs_query = f"""SELECT * FROM programs WHERE stream_id={stream}"""
            df = db4.select(select_programs_query)

            for _, program_name in daily_programs[stream].items():
                if not any(program_name == record['name'] for record in df):
                    missing.add(program_name)

            missing_dict[stream] = missing
        except Exception as e:
            print(f"Error processing stream {stream}: {e}")
            continue

        try:
            for program_name in missing:
                db4.insert(insert_programs_query, (program_name, stream))
        except Exception as e:
            print(f"Error inserting program '{program_name}' for stream {stream}: {e}")

    return missing_dict

def insert_code_to_tv_schedules(daily_programs):
    todays_date = datetime.now().date()
    target_date = todays_date.strftime('%Y-%m-%d')

    for stream in daily_programs:
        try:
            # Check if data already exists for the stream today

            for start_time, program_name in daily_programs[stream].items():
                try:
                    start_datetime = datetime.strptime(f'{target_date} {start_time}', '%Y-%m-%d %H:%M')

                    insert_schedules = """
                        INSERT INTO tv_schedules (start_time, program_id, stream_id)
                        SELECT 
                            %s AS start_time, p.id AS program_id, %s AS stream_id
                        FROM programs p
                        WHERE p.name = %s
                        LIMIT 1;
                    """

                    values = (start_datetime, stream, program_name)
                    db4.insert(insert_schedules, values)
                except Exception as e:
                    print(f"Error inserting program '{program_name}' for stream {stream} at {start_time}: {e}")
        except Exception as e:
            print(f"Error processing stream {stream}: {e}")

In [28]:
# try:
#     for i in daily_programs:
#         daily_programs[i]= clean_dict(daily_programs[i])
# except ValueError as e:
#     print(e)
    
# missing_dict=insert_code_to_programs(daily_programs) 

insert_code_to_tv_schedules(daily_programs)

Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database
Connection to the PostgreSQL database


In [22]:
daily_programs

{'36': {'02:00': 'İtaliya A Seriyası, 14-cü tur: Udineze – Cenoa',
  '05:00': 'İtaliya A Seriyası, 14-cü tur: Fiorentina – İnter',
  '09:30': 'CBC Sport İnfo (canlı)',
  '10:00': 'Ay həkim (premyera)',
  '12:00': 'CBC Sport İnfo (canlı)',
  '12:30': 'Start (canlı)',
  '13:30': 'İtaliya A Seriyası, 14-cü turun oyunlarının icmalı (yeni)',
  '14:30': 'CBC Sport İnfo (canlı)',
  '15:00': 'Futbol, Azərbaycan Kuboku, 1/8 final',
  '17:00': 'CBC Sport İnfo (canlı)',
  '18:00': 'Futbol, Azərbaycan Kuboku, 1/8 final',
  '20:00': 'Misli vaxtı (yeni)',
  '20:15': 'Misli Premyer Liqasının icmalı',
  '21:15': 'İtaliya A Seriyası, 14-cü turun oyunlarının icmalı',
  '23:45': 'Futbol, Azərbaycan Kuboku, 1/8 final'}}

In [27]:
import db4
from datetime import datetime,timedelta


In [32]:
len(daily_programs['36'])

15